### *You must first run the "MISC - Install Libraries.ipynb" notebook to install needed libraries*

## Import your security token from file

In [ ]:
import os
env = %env
token_name = "COPDgene_token.txt"
token_file = token_name
has_error = False
if os.path.isfile(token_file):
    __token__ = open(token_file, "r").read()
    if len(__token__) == 0:
        has_error = True
else: 
    has_error = True
if has_error:
    print("\x1b[31m")
    print("!!! [_Security_Token_Error_] !!!")
    print("Please copy and save a valid PIC-SURE authentication token value into the file \""+token_name+"\".")
    print("This file is located in the current Notebook directory.")
    open(token_file, "w").write("")
else:
    print("\x1b[32m")
    print("[_Security_Token_Imported_Correctly_]")

## Connect to the NHANES data resource using the HPDS Adapter

In [ ]:
import PicSureClient
import PicSureHpdsLib

client = PicSureClient.Client()
connection = client.connect("https://copdgene-dev.hms.harvard.edu/picsure/", __token__, allowSelfSignedSSL=True)
adapter = PicSureHpdsLib.Adapter(connection)
adapter.list()

In [ ]:
resource = adapter.useResource("b6ef7b1a-56f6-11e9-8958-0242c0a83007")

### A "resource" allows access to the Data Dictionary and Query Engine of the underlying  data source

In [ ]:
resource.help()

## Create a new query instance and use .help() to see what can be done with it

In [ ]:
query = resource.query()
query.help()

### Use .help() to see how to add entries to the selection criteria

In [ ]:
query.select().help()

### Use a chained of commands on the dictionary object to include all labs in the query's selection list

In [ ]:
query.select().add(resource.dictionary().find("medication").keys())

### Use the .show() command to see what is currently selected for the query

In [ ]:
query.show()

### Clear the current selection list and add only "\\laboratory\\hormone\\" results

In [ ]:
query.select().clear()
query.select().add(resource.dictionary().find("\\02 Medical history\\Medication history\\").keys())
query.select().show()

### Run some commands on the dictionary to find and then filter the query by gender 

In [ ]:
gender = resource.dictionary().find("gender")
gender.help()

In [ ]:
gender.DataFrame()

### Add a filter on our query to filter results to only female subjects

In [ ]:
query.filter().help()

In [ ]:
query.filter().add("\\01 Demographics\\Gender\\", ["Female"])

## View our query's criterion as it currently stands

In [ ]:
query.show()

### We *MUST* add a consent group that is on the IRB proposal for the query to be accepted

In [ ]:
query.filter().add("\\00 Consent groups\\", ["COPD_HMB"])

### Run the query and display the results

In [ ]:
query.help()

In [ ]:
query.getCount()

In [ ]:
query.getResultsDataFrame()

# You can even work with multiple queries at the same time without having them accidently interact with each other!

In [ ]:
# create 2 different queries
query2 = resource.query()
query3 = resource.query()

# both queries should return the age of the subject at enrollment
query2.select().add(resource.dictionary().find("\\01 Demographics\\Age at enrollment\\").keys())
query3.select().add(resource.dictionary().find("\\01 Demographics\\Age at enrollment\\").keys())

# filter the patient sets using different age cutoffs
query2.filter().add(resource.dictionary().find("\\01 Demographics\\Age at enrollment\\").keys(), min=65)
query3.filter().add(resource.dictionary().find("\\01 Demographics\\Age at enrollment\\").keys(), max=50)

# MUST ADD CONSENT GROUPS TO QUERIES
query2.filter().add("\\00 Consent groups\\", ["COPD_HMB"])
query3.filter().add("\\00 Consent groups\\", ["COPD_HMB"])

# get the results for both queries
too_old = query2.getResultsDataFrame()
too_young = query3.getResultsDataFrame()

In [ ]:
too_young.count()

In [ ]:
too_old.count()